In [136]:


from functions import *
# здесь реализован элементарный генетический алгоритм

In [137]:
# задаем число маяков
nanchors = 4

# число углов комнаты
ncorners = 4

# шаг сетки
stepx = 0.1
stepy = 0.1

# массив координат по х и у
# xcoord = np.array((0.5, 4.5, 3.5, 0.5))
# ycoord = np.array((1.5, 2.5, 4.5, 3.5))

xcoord = np.array((0.5, 3.5, 3.5, 0.5))
ycoord = np.array((0.5, 0.5, 3.5, 3.5))

In [138]:
# формируем комнату
room = createRoom(xcoord, ycoord)
# формируем точки внутри помещения
pointsIn, pointsOut = createPoints(stepx, stepy, xcoord, ycoord)

# нормировка комнаты
roomScaler, pointsInScaler, pointsOutScaler, scaler = standardScalerPoints(xcoord, ycoord, pointsIn, pointsOut)
# нормировка по координатам
xcoordScaler = roomScaler[:, 0]
ycoordScaler = roomScaler[:, 1]

# нормировка обратно inverse_transform(coordAnchors)

In [139]:
# генерируем координаты маяков
# можно не сравнивать со всеми точками внутри - т.к. места маяков не попадают на сетку
coordAnchorsScaler = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)

# подумать нужны ли те точки, которые мы хотим видеть pointsVisible
DOP = estimateDOP(roomScaler, pointsInScaler, coordAnchorsScaler, nanchors, DOPmax=30)

In [140]:
# построение комнаты
plotRoom(roomScaler, pointsInScaler, pointsOutScaler)
plotAnchors(coordAnchorsScaler, roomScaler)
plotDOPfactor(roomScaler, coordAnchorsScaler, DOP, pointsInScaler)

In [141]:
npopul = 50  # Размер популяции
nsurv = 20  # Количество выживших (столько лучших переходит в новую популяцию)
nnew = npopul - nsurv  # Количество новых (столько новых ботов создается)
l = nanchors * 2  # Длина бота
epohs = 20  # Количество эпох
mut = 0.3  # Коэфициент мутаций

In [142]:
popul = []  # Двумерный массив популяции, размерностью [n, l]. 100 ботов по 8 компонентов каждый
val = []  # Одномерный массив значений этих ботов

# Проходим по всей длине популяции
for i in range(npopul):
    bot = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)
    bot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, bot)
    popul.append(bot)

In [143]:
# Создаем пустой список для значений ботов
val = []

# Проходим по всей длине популяции
for bot in popul:
    # Берем очередного бота
    DOP = estimateDOP(roomScaler, pointsInScaler, bot, nanchors, DOPmax=20)
    val.append(sum(DOP) / len(DOP))

In [144]:
# Получаем новую популяцию и сортированный список значнией
newpopul, sval = getSurvPopul(popul, val, nsurv, reverse=0)

# Список выживших - обыкновенным усечением
for i in range(nsurv):
    print(f'бот, {popul[i]},  результат, {sval[i]}')

бот, [[ 0.48429028  0.16636746]
 [ 0.86030564 -0.3508689 ]
 [-0.18467103  0.80088228]
 [-0.56230574 -0.70815613]],  результат, 1.0572996004804291
бот, [[-0.9596766   0.74611685]
 [ 0.85009955 -0.34789399]
 [-0.5671071  -0.64651934]
 [ 0.4510786   0.16150007]],  результат, 1.0806687124575804
бот, [[-0.36757192 -0.50889886]
 [-0.8788313   0.47917245]
 [ 0.86101921  0.7127383 ]
 [-0.75091174 -0.64573028]],  результат, 1.087644682268913
бот, [[ 0.68181841 -0.87710505]
 [-0.16326579  0.7152489 ]
 [ 0.44700656  0.71955901]
 [ 0.68447625  0.05401676]],  результат, 1.0883301815513138
бот, [[-0.357311   -0.13443457]
 [ 0.54153288 -0.5486796 ]
 [-0.11379437  0.87902649]
 [ 0.90546003  0.12045777]],  результат, 1.0934700077173318
бот, [[ 0.19720136 -0.08150323]
 [ 0.74253133 -0.38197704]
 [ 0.81924166 -0.1205596 ]
 [ 0.7386493   0.18502444]],  результат, 1.1030094850102572
бот, [[ 0.21633886  0.04264339]
 [-0.20526005  0.38696426]
 [-0.21358339 -0.82486381]
 [ 0.56953589  0.20244307]],  результат

1. Число особей родителей, участвующих в размножении Q

До этого реализовывалась getParents - функция получения двух родителей, теперь перепишем на то количество родителей, которое мы сами задаем
Здесь появляется свободный параметр ГА Q (getParentsN - число особей-родителей, участвующий в размножении)

3. Размножение

Здесь реализовано равновероятный отбор родителей

При размножении в любом случае должны участвовать разные особи-родители, одного и тогоже родителя нельзя брать более одного раза в одном размножении

Сделано так, чтобы нельзя было брать одиннаковых родителей из выборки

In [145]:
def getParentsN(nParentReproduct, currPopul, nsurv):
    '''
    функция получения родителей
    :param nParentReproduct:
    :param currPopul:
    :param nsurv:
    :return:
    '''
    botp = []
    nsurvi = [i for i in range(nsurv)]
    for parent in range(nParentReproduct):
        indexp = random.choice(nsurvi)
        botp.append(currPopul[indexp])
        nsurvi.remove(indexp)
    return botp

In [146]:
nParentReproduct = 3

In [147]:
parents = getParentsN(nParentReproduct, newpopul, nsurv)
for i in range(nParentReproduct):
    print('Родитель {}, {}'.format(i, parents[i]))

Родитель 0, [[-0.357311   -0.13443457]
 [ 0.54153288 -0.5486796 ]
 [-0.11379437  0.87902649]
 [ 0.90546003  0.12045777]]
Родитель 1, [[ 0.9117302   0.87535788]
 [-0.2959566  -0.5954393 ]
 [ 0.04413914  0.92908951]
 [ 0.58293092  0.05469845]]
Родитель 2, [[-0.61598174 -0.78222677]
 [ 0.06128288  0.43783175]
 [ 0.32680266  0.90070029]
 [ 0.88927048 -0.33446923]]


5. Мутации

Добавить мутации после размножения

Удаление-вставка, замена старого гена новым
Причем это для каждого родителя - циклом по три
1. Нужно определить случайное число генов, которые подлежат замене с помощью операции генерации случайного числа с нормальным законом распределения, нулевым средним и СКО, которое задаем как входной параметр ГА (как ставить ограничение на это число, чтобы если что оно не вылетало за пределы длины бота?)
2. Далее определить порядковый номер заменяемого гена - случайный равновероятный способ по длине бота (от нуля до длины бота)

In [148]:
skoGenesReplace = 2  # ско чила заменяемых генов

И если это число больше нуля, то идём в цикл
Где соответственно генерируем столько индексов бота, сколько было сгенерировано выше

In [149]:
genesReplace = abs(round(skoGenesReplace * np.random.randn()))
if genesReplace > 0:
    for genes in range(genesReplace):
        indexReplace = random.randint(0, l - 1)
        print('ген родителя 1', parents[0].reshape(nanchors * 2)[indexReplace])

Новое значение подбирается случайным образом по нормальному закону, центрированному относительно старого значения

Длина помещения по заданной координате? И как мы можем знать заранее какая это координата и как это подставлять?
Если четное? берем такую длину, если нечетный индекс другую?
Пока оставлю только СКО

In [150]:
skoCoordVariation = 0.25  # ско вариации координаты

In [151]:
# удаление вставка - старого гена новым

for parent in parents:
    genesReplace = abs(round(skoGenesReplace * np.random.randn()))
    if genesReplace > 0:
        for genes in range(genesReplace):
            indexReplace = random.randint(0, l - 1)
            parent.reshape(nanchors * 2)[indexReplace] += skoCoordVariation * np.random.randn()

In [ ]:
# это уже не в родителях а отдельно по боту добавление после

Обмен хромосомами.
1. Случайным образом, как в предыдущем пункте определяется число пар генов, которые подлежат замене - максимум - в два раза меньше длины
2. Далее просто равновероятно определяем какие гены должны поменяться в боте

In [152]:
skoChangeGenes = 1.25  # ско числа обменивающихся пар генов

Сколько пар поменять местами и как именно мы выбираем пары, которые менять

In [153]:
# обмен генами в хромосоме
for parent in parents:
    genesChange = abs(round(skoChangeGenes * np.random.randn()))
    if genesChange > 0:
        for genes in range(genesChange):
            while 1:
                index1 = random.randint(0, l - 1)
                index2 = random.randint(0, l - 1)
                if index1 != index2:
                    break
            value1 = parent.reshape(nanchors * 2)[index1]
            value2 = parent.reshape(nanchors * 2)[index2]
            parent.reshape(nanchors * 2)[index1] = value2
            parent.reshape(nanchors * 2)[index2] = value1

Реализация размножения/скрещивания

Число точек разрыва хромосомы R

Число точек разрыва хромосомы R.  Хромосомы Q родителей разрываются на R+1 частей в одинаковых местах, выбираемых случайно равновероятно. Хромосома потомка набирается случайным равновероятным образом из соответствующих фрагментов хромосом родителей.

При каждом размножении получается A потомков, хромосомы которых собираются независимо друг от друга. (При A>1 есть вероятность случайно собрать близнецов с одинаковыми хромосомами.)

Сейчас реализовано равномерное скрещивание

In [256]:
def crossPointFromNParents(nParentReproduct, parents, nanchors, lenbot):
    '''
    функция смешивания (кроссинговера) нескольких родителей
    :param nParentReproduct:
    :param botp:
    :param idx:
    :return:
    '''
    bot = np.zeros(lenbot)
    for i in range(lenbot):
        idx = random.randint(0, nParentReproduct - 1)
        parent = parents[idx].reshape(l)
        bot[i] = parent[i]
    return bot.reshape((nanchors, 2))

In [257]:
bot = crossPointFromNParents(nParentReproduct, parents, nanchors, l)

In [258]:
bot

array([[-0.61598174, -0.13443457],
       [ 0.05776414, -0.5486796 ],
       [ 0.92908951,  0.87902649],
       [ 0.90546003,  0.05469845]])

Требуется сделать разбивку на хромосомы, и на части

In [252]:
nPartPoints = 3 # число точек разрыва хромосомы при размножении

In [253]:
def partPointFromNParents(nParentReproduct, parents, nanchors, lenbot, nPartPoints):
    '''
    функция смешивания (кроссинговера) нескольких родителей
    :param nParentReproduct:
    :param botp:
    :param idx:
    :return:
    '''
    bot = np.zeros(lenbot)
    def partDivision(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i: i + n]
    partGenes = list(partDivision([i for i in range(lenbot)], round(lenbot / (nPartPoints + 1))))
    for part in partGenes:
        idx = random.randint(0, nParentReproduct - 1)
        parent = parents[idx].reshape(lenbot)
        bot[part] = parent[part]
    return bot.reshape((nanchors, 2))

In [255]:
bot = partPointFromNParents(nParentReproduct, parents, nanchors, l, nPartPoints)

In [ ]:
for i in range(nsurv):
    print(f'бот, {popul[i]},  результат, {sval[i]}')

    # Проходимся в цикле nnew-раз
    for i in range(nnew):

        # Из newpopul(новой популяции) получаем двух случайных родителей-ботов
        botp1, botp2 = getParents(newpopul, nsurv)
        # Массив для нового бота
        newbot = []

        # проходимся по длине бота и осуществляем смешивание/скрещивание от родителей
        for j in range(l):
            # Получаем значение для j-ого компонента бота
            x = crossPointFrom2Parents(botp1.reshape(l), botp2.reshape(l), j)
            # Добавляем к значению бота случайную величину, зависящую от коэфециента мутации
            x += mut * (2 * random.random() - 1.0)
            # Добавляем новое значение в бота
            newbot.append(x)  # Добавялем новое значение в бота

        # Добавляем бота в новую популяцию
        newbot = np.array(newbot).reshape((nanchors, 2))
        newbot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, newbot)
        newpopul.append(newbot)

        #(таким образом к nsurv-лучших ботов предыдующей популяции добавится nnew-новых ботов)

    # Записываем в popul посчитанную новую популяцию
    popul = newpopul